In [ ]:
#Starting with the baseline Google Contrail Dataset file of the format json-00000-of-00100, shortened to filename where the 000xx matches the original file name of the 000xx of 00100 format
import json
count=0
with open('/root/file00012.json') as f:  #https://stackoverflow.com/questions/21533894/how-to-read-line-delimited-json-from-large-file-line-by-line
    for line in f:
        j_content = json.loads(line)
        count=count+1
        outfile = "outfile00012_" + str(count) + ".json" #https://www.geeksforgeeks.org/reading-and-writing-json-to-a-file-in-python/
        with open(outfile, "w") as outfile:
            json.dump(j_content, outfile)
            
#Result is the listing of individual files broken from the original google source, typically 40 or so files embedded within the original Google bucket download file

In [ ]:
pwd()

In [ ]:
#This will read the files in a list
import os
import numpy as np
bbox_path = '/root/contrail_json'
json_files = [f for f in os.listdir(bbox_path) if f.endswith('.json')] #https://stackoverflow.com/questions/3964681/find-all-files-in-a-directory-with-extension-txt-in-python

In [ ]:
#From Landsat_Contrails_Dataset original .ipynb file
def path_from_filename(landsat_filename):
  """Returns the full GCP filepath for the provided `landsat_filename`.

  An example filename looks like:
  LC08_L1TP_221074_20181218_20181227_01_T1_B10.TIF

  The returned path ends up looking like:
  gs://gcp-public-data-landsat/LC08/01/221/074/LC08_L1TP_221074_20181218_20181227_01_T1/LC08_L1TP_221074_20181218_20181227_01_T1_B10.TIF

  Args:
    landsat_filename: str, file name of a landsat scene.
  Returns:
    str, full path to the corresponding landsat scene.
  """
  split_name = landsat_filename.split('_')
  path, row = split_name[2][:3], split_name[2][3:]
  bands = split_name[0]  # Either LC08 or LT08
  parent_dir = '_'.join(split_name[:-1])
  return os.path.join(f'gs://gcp-public-data-landsat/{bands}/01/',
                      path, row, parent_dir, landsat_filename)

_ = """
Copyright 2021 The Landsat Contrails Dataset Authors.

Licensed under the Apache License, Version 2.0 (the "License");                                                         
you may not use this file except in compliance with the License.                                                        
You may obtain a copy of the License at                                                                                 
                                                                                                                        
    https://www.apache.org/licenses/LICENSE-2.0                                                                         
                                                                                                                        
Unless required by applicable law or agreed to in writing, software                                                     
distributed under the License is distributed on an "AS IS" BASIS,                                                       
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.                                                
See the License for the specific language governing permissions and                                                     
limitations under the License.
"""

In [ ]:
!pip install gsutil

In [ ]:
#From DrawBoundingBoxesOnImage.ipynb from Summer Group

#This will porse the contrails from each individual contrail dataset reference, if >0 download images and place in a Positive folder, otherwise negative

import os
import numpy as np
bbox_path = '/root/contrail_json'
yolo_path = '/root/contrail_processed'

for filename in json_files:
  f = open(os.path.join(bbox_path,filename))
  indiv_json_file = json.loads(f.read()) # reads file in as a python object
  list_of_polygons = indiv_json_file['polygons']
  num_of_contrails_infile = (len(list_of_polygons))
  current_contrail = 0
  image_size = 1400
  classifier = np.array([0])
  image_file = indiv_json_file['filename']

  while current_contrail < num_of_contrails_infile:
    print(list_of_polygons[current_contrail][-1])  #verification
    if len(list_of_polygons[current_contrail][-1]) < 4:
      print(filename)
      current_contrail = current_contrail + 1
      continue
    #perform swap
    x2 = list_of_polygons[current_contrail][-1][1]
    y1 = list_of_polygons[current_contrail][-1][2] 
    temp = copy.deepcopy(list_of_polygons[current_contrail][-1][1]) # store x2 in temp
    x2 = copy.deepcopy(y1)
    y1 = copy.deepcopy(temp)
    list_of_polygons[current_contrail][-1][1] = x2
    list_of_polygons[current_contrail][-1][2] = y1
    print(list_of_polygons[current_contrail][-1]) # verification

    fileType = '.txt'
    textfile_name_list = filename.split(".") 
    textfile_name = textfile_name_list[0]+ fileType

    contrail1 = np.array(list_of_polygons[current_contrail][-1]) # convert list to a numpy arry
    contrail1 = contrail1/image_size  # normalize bounding box
    print(contrail1) # verification 
    y = open(os.path.join(yolo_path,textfile_name), 'a') # open text file
    classifier = list(classifier)
    contrail1 = list(contrail1)
    for num in classifier:
      y.write(str(num))
      y.write(" ")
    for number in contrail1:
      y.write(str(number))
      y.write(" ")
    y.write(" \n")
    current_contrail = current_contrail + 1
    
    print(" ")
  print(current_contrail)
  print(str(current_contrail) + ': ' + filename + " " + image_file)
  if current_contrail==0:
    folder='Neg'
  else:
    folder='Pos'
  filename_11um = image_file
  filename_12um = filename_11um.replace('B10.TIF', 'B11.TIF')
  filename_1370nm = filename_11um.replace('B10.TIF', 'B9.TIF')
  filename_red =  filename_11um.replace('B10.TIF', 'B4.TIF')
  filename_green =  filename_11um.replace('B10.TIF', 'B3.TIF')
  filename_blue =  filename_11um.replace('B10.TIF', 'B2.TIF')
  filename_mtl = filename_11um.replace('B10.TIF', 'MTL.txt')

  !gsutil cp {path_from_filename(filename_11um)} /root/contrail_processed/{folder}/
  !gsutil cp {path_from_filename(filename_12um)} /root/contrail_processed/{folder}/
  !gsutil cp {path_from_filename(filename_1370nm)} /root/contrail_processed/{folder}/
  !gsutil cp {path_from_filename(filename_red)} /root/contrail_processed/{folder}/
  !gsutil cp {path_from_filename(filename_green)} /root/contrail_processed/{folder}/
  !gsutil cp {path_from_filename(filename_blue)} /root/contrail_processed/{folder}/
  !gsutil cp {path_from_filename(filename_mtl)} /root/contrail_processed/{folder}/

  path_11um = os.path.join('/root/contrail_processed/{folder}/', filename_11um)
  path_12um = os.path.join('/root/contrail_processed/{folder}/', filename_12um)
  path_1370nm = os.path.join('/root/contrail_processed/{folder}/', filename_1370nm)
  path_red = os.path.join('/root/contrail_processed/{folder}/', filename_red)
  path_green = os.path.join('/root/contrail_processed/{folder}/', filename_green)
  path_blue = os.path.join('/root/contrail_processed/{folder}/', filename_blue)
  path_mtl = os.path.join('/root/contrail_processed/{folder}/', filename_mtl)

In [ ]:
#Convert to .JPG
#Note, 5 times faster on GPU ...about 2 seconds per image vs. 0 otherwise

In [ ]:
import os
tif_files = [f for f in os.listdir('/root/contrail_processed/Pos') if f.endswith('.TIF')]

In [ ]:
os.chdir('/root/contrail_processed/Pos')

In [ ]:
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
for filename in tif_files:
    im=Image.open(filename)
    new_jpeg = filename.replace("TIF", "JPG") #https://www.w3schools.com/python/ref_string_replace.asp
    plt.imsave(new_jpeg, im)

In [ ]:
import os
tif_files = [f for f in os.listdir('/root/contrail_processed/Neg') if f.endswith('.TIF')]

In [ ]:
os.chdir('/root/contrail_processed/Neg')

In [ ]:
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
for filename in tif_files:
    im=Image.open(filename)
    new_jpeg = filename.replace("TIF", "JPG") #https://www.w3schools.com/python/ref_string_replace.asp
    plt.imsave(new_jpeg, im)

In [ ]:
#AWS CLI examples copying to S3 buckets

In [ ]:
!aws s3 cp s3://contrail-classification-input-101522/neg/ s3://contrail-classification-input-b4/neg/ --recursive --exclude "*" --include "*B4*" 
#https://stackoverflow.com/questions/38834708/how-can-i-use-wildcards-to-cp-a-group-of-files-with-the-aws-cli

In [ ]:
!aws s3 cp s3://contrail-classification-input-101022/neg/ s3://contrail-classification-input-b4/neg/ --recursive --exclude "*" --include "*B4*" 
#https://stackoverflow.com/questions/38834708/how-can-i-use-wildcards-to-cp-a-group-of-files-with-the-aws-cli

In [ ]:
!aws s3 cp s3://contrail-classification-input-101522/pos/ s3://contrail-classification-input-b4/pos/ --recursive --exclude "*" --include "*B4*" 

In [ ]:
!aws s3 cp s3://contrail-classification-input-101022/pos/ s3://contrail-classification-input-b4/pos/ --recursive --exclude "*" --include "*B4*" 